In [1]:
import jupyter_black
import numpy as np
import pandas as pd
from IPython.display import clear_output
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier

In [2]:
jupyter_black.load(lab=False, line_length=79)

<IPython.core.display.Javascript object>

# Задание 1
Сколько суммарно пропусков в данных?

In [3]:
df = pd.read_csv("weatherAUS.csv", parse_dates=["Date"])

In [4]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [5]:
df.isna().sum().sum()

343248

# Задание 2
В некоторых признаках пропусков более 40 % — удалите такие признаки. Сколько их было?

In [6]:
columns_to_delete = [
    column for column in df.columns if df[column].isna().sum() > 0.4 * len(df)
]

In [7]:
columns_to_delete

['Evaporation', 'Sunshine', 'Cloud3pm']

In [8]:
df = df.drop(columns=columns_to_delete)

# Задание 3
Теперь обработаем признаки RainToday и RainTomorrow таким образом, чтобы вместо yes было значение 1, а вместо no — значение 0.

In [9]:
df["RainToday"] = df["RainToday"].map({"No": 0, "Yes": 1})
df["RainTomorrow"] = df["RainTomorrow"].map({"No": 0, "Yes": 1})

In [10]:
df["RainToday"].mean()

0.22419285648984874

# Задание 4
Обработайте признак Date таким образом, чтобы выделить в отдельный признак Month (номер месяца). Изначальный признак Date удалите. Определите, в какой месяц выпадает больше всего дождей.

In [11]:
df["Month"] = df["Date"].dt.month

In [12]:
df = df.drop(columns="Date")

In [13]:
df.groupby("Month").mean(numeric_only=True).sort_values(
    "RainToday", ascending=False
)

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Temp9am,Temp3pm,RainToday,RainTomorrow
Month,,,,,,,,,,,,,,,
7,6.944633,16.744580,2.184209,37.963884,12.977923,17.347654,77.488055,58.005207,1021.121071,1018.730487,4.454439,10.830630,15.631336,0.270736,0.269208
6,7.818067,17.307648,2.782182,35.560891,12.293303,15.870799,79.406630,60.540855,1021.200465,1018.879716,4.744507,11.565653,16.186395,0.263638,0.261845
8,7.459653,18.241013,2.034983,40.280165,13.868044,18.911306,71.420059,52.816025,1019.603766,1016.991759,4.094023,12.372772,16.951018,0.253167,0.252508
9,9.466875,20.752012,1.888543,42.294806,15.478037,20.175620,64.944760,50.334514,1018.039954,1015.251349,4.131386,15.350247,19.188121,0.229135,0.229705
5,9.640925,20.065173,2.001654,35.803096,12.420218,16.356048,74.675386,55.187292,1020.449787,1017.949934,4.459665,14.116108,18.822236,0.222163,0.225278
3,15.913305,26.885273,2.814450,39.599356,13.332753,18.466672,70.816985,50.357165,1016.766232,1014.551846,4.490458,19.888484,25.343573,0.217135,0.213332
4,12.843618,23.625035,2.334823,36.505243,12.877838,17.153968,70.803854,52.219453,1019.641567,1017.043485,4.496956,17.680004,22.239993,0.216845,0.217849
12,15.811322,27.558321,2.491835,43.071686,15.274989,20.215366,61.568847,47.339779,1013.317432,1011.109731,4.667516,21.225267,25.612047,0.213037,0.208141
11,14.321606,26.181247,2.268177,42.617647,15.238240,20.080046,61.787145,47.889799,1015.385387,1013.041614,4.637376,19.970794,24.272204,0.210843,0.212547


# Задание 5
Обработайте оставшиеся категориальные признаки. С помощью метода get_dummies с настройками по умолчанию создайте dummy-переменные для всех категориальных признаков (их пять), которые есть в данных на этот момент.

In [14]:
df = pd.get_dummies(
    df,
    columns=["Month", "Location", "WindGustDir", "WindDir9am", "WindDir3pm"],
)

In [15]:
df.shape

(145460, 124)

# Задание 6
Осталось совсем немного. Удалите все строки, где есть пропуски. Далее разбейте данные на обучающую и тестовую выборки в соотношении 70/30, в качестве значения параметра `random_state` возьмите число 31.

In [16]:
df = df.dropna()

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns="RainTomorrow"),
    df["RainTomorrow"],
    test_size=0.3,
    random_state=31,
)

In [18]:
np.mean(y_test)

0.22770253002811142

# Задание 7
Теперь давайте вспомним про бутстреп. Он не понадобится нам для решения этой задачи, но будет полезно реализовать его «вручную».

Сделайте оценку стандартного отклонения для среднего значения минимальной температуры для обучающей выборки (то есть для среднего значения по признаку MinTemp). Для этого сгенерируйте 1000 случайных выборок из наших данных — каждая из них должна быть такого же объёма, как и обучающая выборка. Для генерации выборки используйте `np.random.randint()`: сгенерируйте необходимое количество индексов и по ним извлеките соответствующие элементы выборки. Случайность фиксируйте с помощью `np.random.seed(31)`.

Для каждой выборки вычислите среднее значение, а после найдите стандартное отклонение для этих значений.

In [19]:
np.random.seed(31)

In [20]:
samples = X_train["MinTemp"].values[
    np.random.randint(0, len(X_train), (1000, len(X_train)))
]

In [21]:
round(np.std(np.mean(samples, axis=1)), 2)

0.03

# Задание 8
Теперь можно перейти к обучению прогностических моделей. Начнём с того, что построим простейшую логистическую регрессию (без настройки гиперпараметров). Это будет та модель, с качеством которой мы будем сравнивать результаты, полученные далее, чтобы оценить превосходство случайного леса над простыми методами.

In [22]:
clf = LogisticRegression()

In [23]:
clf.fit(X_train, y_train)
clear_output()

In [24]:
y_pred = clf.predict(X_test)

In [25]:
round(roc_auc_score(y_test, y_pred), 2)

0.73

# Задание 9
Теперь попробуйте обучить на наших данных другой алгоритм — дерево решений. С помощью `GridSearchCV` сделайте перебор гиперпараметров по следующей сетке:

In [26]:
param_grid = {
    "max_depth": [5, 7, 9, 11],
    "min_samples_split": [2, 3, 4],
    "random_state": [42],
    "max_leaf_nodes": list(range(2, 10)),
}

Для параметра кросс-валидации `cv` задайте значение 3. Для решающего дерева определите параметр `random_state=42`. Остальные параметры оставьте по умолчанию.

In [27]:
clf = DecisionTreeClassifier(random_state=42)

In [28]:
cv = GridSearchCV(clf, param_grid, n_jobs=-1, cv=3, verbose=3)

In [29]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 7, 9, 11],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_split': [2, 3, 4], 'random_state': [42]},
             verbose=3)

In [30]:
cv.best_params_

{'max_depth': 5,
 'max_leaf_nodes': 9,
 'min_samples_split': 2,
 'random_state': 42}

In [31]:
clf = DecisionTreeClassifier(
    max_depth=5, min_samples_split=2, random_state=42, max_leaf_nodes=9
)

In [32]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, max_leaf_nodes=9, random_state=42)

In [33]:
y_pred = clf.predict(X_test)

In [34]:
round(roc_auc_score(y_test, y_pred), 2)

0.7

# Задание 10
К сожалению, деревья решений не помогли нам в улучшении качества модели, так что попробуем ещё уменьшить ошибку с помощью ансамблей.

Теперь постройте случайный лес, включающий 100 деревьев. Задайте параметр `random_state=31`. Остальные параметры оставьте по умолчанию.

In [35]:
clf = RandomForestClassifier(random_state=31)

In [36]:
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=31)

In [37]:
y_pred = clf.predict(X_test)

In [38]:
round(roc_auc_score(y_test, y_pred), 2)

0.73

# Задание 11
Основные параметры, которые отвечают за качество обучения в случайном лесе, следующие: `max_features`, `min_samples_leaf`, `max_depth`.

Возьмите случайный лес из 100 деревьев и найдите оптимальную комбинацию этих трёх параметров. Сетка для перебора следующая:

In [39]:
param_grid = {
    "max_depth": [5, 10, 15],
    "min_samples_leaf": [3, 5, 7, 9, 11],
    "max_features": [4, 5, 6, 7],
    "random_state": [31],
}

Перебор осуществите с помощью `GridSearchCV`. Для параметра кросс-валидации `cv` задайте значение 3. Случайности фиксируйте параметром `random_state=31`. Остальные значения оставьте по умолчанию.

In [40]:
clf = RandomForestClassifier()

In [41]:
cv = GridSearchCV(clf, param_grid, n_jobs=-1, cv=3, verbose=3)

In [42]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15], 'max_features': [4, 5, 6, 7],
                         'min_samples_leaf': [3, 5, 7, 9, 11],
                         'random_state': [31]},
             verbose=3)

In [43]:
cv.best_params_

{'max_depth': 15, 'max_features': 7, 'min_samples_leaf': 3, 'random_state': 31}

In [44]:
clf = RandomForestClassifier(
    max_depth=15, min_samples_leaf=3, max_features=7, random_state=31
)

In [45]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=15, max_features=7, min_samples_leaf=3,
                       random_state=31)

In [46]:
y_pred = clf.predict(X_test)

In [47]:
round(roc_auc_score(y_test, y_pred), 2)

0.7

# Задание 12
Как мы говорили в предыдущем юните, благодаря случайному лесу можно узнать, какие признаки оказывают большее влияние на целевую переменную по сравнению с другими. Оцените значимость признаков.

In [48]:
pd.DataFrame(
    {
        "feature": df.drop(columns="RainTomorrow").columns,
        "importance": clf.feature_importances_,
    }
).sort_values(by="importance", ascending=False).head(3)

,feature,importance
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
